In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
image_size = [224,224]
train_loc = '../input/chest-xray-pneumonia/null'
test_loc = '../input/chest-xray-pneumonia/null'
valid_loc = '../input/chest-xray-pneumonia/null'

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet50 = ResNet50(input_shape=image_size+[3],weights='imagenet',include_top=False)

In [ ]:
resnet50.summary()

In [ ]:
for layer in resnet50.layers:
    layer.trainable=False

In [ ]:
from glob import glob
folders = glob('../input/chest-xray-pneumonia/chest_xray/train/*')
len(folders)

In [ ]:
from tensorflow.keras.layers import Flatten
X=Flatten()(resnet50.output)

In [ ]:
from tensorflow.keras.layers import Dense
pred = Dense(2,activation='softmax')(X)

In [ ]:
from tensorflow.keras.models import Model
model = Model(inputs=resnet50.input,outputs=pred)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_dataaug = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True
                                  )

test_dataaug = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_dataaug.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train',target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
test_set = test_dataaug.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',target_size=(224,224),batch_size=32,class_mode='categorical')

In [ ]:
model.fit(train_set,validation_data=test_set,epochs=2)

In [ ]:
y_pred = model.predict(test_set)
y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred,axis=1)
y_pred

In [ ]:
from tensorflow.keras.preprocessing import image
img = image.load_img('../input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg',target_size=(224,224))

In [ ]:
x= image.img_to_array(img)
x

In [ ]:
x = x/255
x.shape

In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
x=np.expand_dims(x,axis=0)
img_data = preprocess_input(x)

In [ ]:
model.predict(img_data)

In [ ]:
p = np.argmax(model.predict(img_data),axis=1)
if p==1:
    print("uninfected")
else:
    print("Infected")
          